In [9]:
from pathlib import Path
import cv2
import numpy as np
import os

source_dir = Path("../data/pzl_3")
target_dir = Path("../data/pzl_3_processed")
target_dir.mkdir(parents=True, exist_ok=True)
target_size = (256, 256)

def crop_and_pad_to_square(image, output_size=(256, 256), threshold=90):
    """Beschneidet das Objekt im Bild (weißem Hintergrund) und zentriert es auf einem quadratischen Canvas"""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY_INV)
    coords = cv2.findNonZero(mask)
    if coords is None:
        return cv2.resize(image, output_size)  # kein Objekt erkannt

    x, y, w, h = cv2.boundingRect(coords)
    cropped = image[y:y+h, x:x+w]

    # Resizing mit Seitenverhältnis
    aspect_ratio = w / h
    if aspect_ratio > 1:  # breiter als hoch
        new_w = output_size[0]
        new_h = int(output_size[1] / aspect_ratio)
    else:
        new_h = output_size[1]
        new_w = int(output_size[0] * aspect_ratio)
    
    resized = cv2.resize(cropped, (new_w, new_h), interpolation=cv2.INTER_LANCZOS4)

    # Zentriert auf weißem Hintergrund einfügen
    result = np.full((output_size[1], output_size[0], 3), 255, dtype=np.uint8)
    x_offset = (output_size[0] - new_w) // 2
    y_offset = (output_size[1] - new_h) // 2
    result[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized
    print(f"Originalgröße: {image.shape}, Cropgröße: {cropped.shape}, Resize: ({new_w}, {new_h})")

    return result

for img_path in sorted(source_dir.glob("*.jpg")):
    img = cv2.imread(str(img_path))
    if img is None:
        print(f" Fehler beim Laden: {img_path.name}")
        continue
    
    cropped = crop_and_pad_to_square(img)
    processed = cv2.resize(cropped, target_size, interpolation=cv2.INTER_AREA)

    # Speichere die Datei im Zielordner mit dem gleichen Namen
    target_file = target_dir / img_path.name
    cv2.imwrite(str(target_file), processed)

    print(f"✔️ {img_path.name} verarbeitet und gespeichert nach {target_file}")

Originalgröße: (4640, 3472, 3), Cropgröße: (2864, 2178, 3), Resize: (194, 256)
✔️ pzl_3_001_back.jpg verarbeitet und gespeichert nach ../data/pzl_3_processed/pzl_3_001_back.jpg
Originalgröße: (4640, 3472, 3), Cropgröße: (1273, 1302, 3), Resize: (256, 250)
✔️ pzl_3_001_front.jpg verarbeitet und gespeichert nach ../data/pzl_3_processed/pzl_3_001_front.jpg
Originalgröße: (4640, 3472, 3), Cropgröße: (2866, 2120, 3), Resize: (189, 256)
✔️ pzl_3_002_back.jpg verarbeitet und gespeichert nach ../data/pzl_3_processed/pzl_3_002_back.jpg
Originalgröße: (4640, 3472, 3), Cropgröße: (1008, 1644, 3), Resize: (256, 156)
✔️ pzl_3_002_front.jpg verarbeitet und gespeichert nach ../data/pzl_3_processed/pzl_3_002_front.jpg
Originalgröße: (4640, 3472, 3), Cropgröße: (2737, 2387, 3), Resize: (223, 256)
✔️ pzl_3_003_back.jpg verarbeitet und gespeichert nach ../data/pzl_3_processed/pzl_3_003_back.jpg
Originalgröße: (4640, 3472, 3), Cropgröße: (1380, 1468, 3), Resize: (256, 240)
✔️ pzl_3_003_front.jpg verarbeit